In [3]:
import os 
from dotenv import load_dotenv

In [4]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


Setting the environment key

In [5]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [6]:
from langchain.document_loaders import PyPDFLoader

flutter_dir = "../Resource/flutter.pdf"
loader = PyPDFLoader(flutter_dir)
data = loader.load()


In [7]:
len(data)

185

In [8]:
question_gen = "" 
for page in data:
    question_gen += page.page_content

In [9]:
print(question_gen)

ii 
 
About the T utorial 
Flutter is an open source framework to create high quality, high performance mobile 
applications across mobile operating system s - Android and iOS. It provides a simple, 
powerful, efficient and easy to understand SDK to write mobile application in Google’s own 
language, Dart.  
This tutorial walks through the basics of Flutter framework, install ation of Flutter SDK, 
setting up Android Studio to develop Flutter based application, architecture of Flutter 
framework and developing all type of mobile applications using Flutter framework. 
Audience 
This tutorial is prepared for professionals who are aspiring to make a career in the field of 
mobile applications. This tutorial is intended to make you comfortable in getting started 
with Flutter framework and its various functionalities. 
Prerequisites 
This tutorial is written  assuming that the readers are already aware about what a 
Framework is  and that the readers have a sound knowledge on Object Orient

In [10]:
from langchain.text_splitter import  TokenTextSplitter

#here we will use the LLM for splitting
splitter_ques_gen = TokenTextSplitter(
    model_name="gpt-3.5-turbo",
    chunk_size = 1000,
    chunk_overlap = 200
)

In [11]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)

In [12]:
len(chunk_ques_gen)

70

In [13]:
from langchain.docstore.document import Document

document_ques_gen = [Document(page_content=t) for t in chunk_ques_gen]
document_ques_gen

[Document(metadata={}, page_content='ii \n \nAbout the T utorial \nFlutter is an open source framework to create high quality, high performance mobile \napplications across mobile operating system s - Android and iOS. It provides a simple, \npowerful, efficient and easy to understand SDK to write mobile application in Google’s own \nlanguage, Dart.  \nThis tutorial walks through the basics of Flutter framework, install ation of Flutter SDK, \nsetting up Android Studio to develop Flutter based application, architecture of Flutter \nframework and developing all type of mobile applications using Flutter framework. \nAudience \nThis tutorial is prepared for professionals who are aspiring to make a career in the field of \nmobile applications. This tutorial is intended to make you comfortable in getting started \nwith Flutter framework and its various functionalities. \nPrerequisites \nThis tutorial is written  assuming that the readers are already aware about what a \nFramework is  and tha

In [14]:
%pip install langchain-google-genai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001",api_key=GOOGLE_API_KEY)


In [16]:
prompt_template = """
You are a expert at creating accurate interview questions based on the coding materilas form flutter  and  its documentations 
your goal is to preapre a coder  or programmer for their exam and coding test, you do
 this by taking questions  about the text below  and creating a interview question based on the text.

 -----------------------------
 {text}
 -----------------------------

 create questions that will prepeare the coders or programmer for their interview and give their best in the interview  dont leave any important information to be
  covered in the questions that is really important

 


QUESTIONS:
"""

In [17]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=["text"],template=  prompt_template)

In [36]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------
Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original question. final response should only contains question nothing else each question should be seperated by new line character
QUESTIONS:
"""
)


In [37]:
refine_prompt_template = PromptTemplate(
    input_variables=["existing_asnwer","text"],
    template= refine_template,
)

# HOW THIS WORKS?

### Scenario Setup
Suppose you have 5 document chunks about Flutter:

Doc 1: Introduction to Flutter

Doc 2: Flutter Widgets

Doc 3: State Management in Flutter

Doc 4: Navigation in Flutter

Doc 5: Flutter Testing

Your goal: Generate a comprehensive set of interview questions covering all these topics.

Step-by-Step Chain Execution
### Step 1: First Document (Initial Prompt)
        Input:
        Doc 1: "Flutter is an open-source UI toolkit for building natively compiled applications for mobile, web, and desktop from a single codebase."

        Prompt Used:
        prompt_template (asks to generate interview questions based on the text)

        LLM Output:

        What is Flutter?

        What platforms does Flutter support?

        What is the main advantage of using Flutter?

### Step 2: Second Document (Refine Prompt)
        Input:

        Existing questions (from Step 1)

        Doc 2: "Flutter widgets are the basic building blocks of a Flutter app’s user interface."

        Prompt Used:
        refine_template (asks to refine or add questions based on new context)

        LLM Output:

        What is Flutter?

        What platforms does Flutter support?

        What is the main advantage of using Flutter?

        What are Flutter widgets?

        How do widgets contribute to building a UI in Flutter?

### Step 3: Third Document (Refine Prompt)
            Input:

            Existing questions (from Step 2)

            Doc 3: "State management refers to the management of the state of one or more user interface controls."

            Prompt Used:
            refine_template

            LLM Output:

            What is Flutter?

            What platforms does Flutter support?

            What is the main advantage of using Flutter?

            What are Flutter widgets?

            How do widgets contribute to building a UI in Flutter?

            What is state management in Flutter?

            Why is state management important in Flutter apps?

### Step 4: Fourth Document (Refine Prompt)
        Input:

        Existing questions (from Step 3)

        Doc 4: "Navigation in Flutter is managed by Navigator and Routes."

        Prompt Used:
        refine_template

        LLM Output:

        (All previous questions)

        What is navigation in Flutter?

        How does the Navigator widget work?

        What are Routes in Flutter?

### Step 5: Fifth Document (Refine Prompt)
        Input:

        Existing questions (from Step 4)

        Doc 5: "Flutter provides a testing framework for unit, widget, and integration tests."

        Prompt Used:
        refine_template

        LLM Output (Final):

        (All previous questions)

        What types of testing does Flutter support?

        How do you write a unit test in Flutter?

        What is the purpose of widget testing?

In [38]:
from langchain.chains.summarize import load_summarize_chain


ques_gen_chain = load_summarize_chain(llm = llm,chain_type="refine",
                                      question_prompt = prompt_template,
                                      refine_prompt = refine_prompt_template)


In [39]:
final_questions = ques_gen_chain.run(document_ques_gen[:3])

In [40]:
print(final_questions)

What is Flutter and what problem does it solve?
What are the key advantages of using Flutter for mobile app development, according to the tutorial?
What programming language is used in Flutter, and what prerequisites does the tutorial mention for someone learning Flutter?
Flutter is described as an open-source framework. What does open-source mean in this context, and why is it significant?
Explain the architecture of a Flutter application. What are the key layers involved?
What are widgets in Flutter? Give some examples of different types of widgets.
Explain the concept of "state" in Flutter. Why is state management important in Flutter applications?
What is the role of Gestures in Flutter and why is it important for application development?
Describe the difference between Single-child and multi-child layout widgets. Give examples of each.
Name some Single Child Widgets and explain what they do?
Name some Multiple Child Widgets and explain what they do?
Explain the difference between 

In [41]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)

In [42]:
from langchain.vectorstores import FAISS
vector_store= FAISS.from_documents(document_ques_gen,embedding=embeddings)
  

In [43]:
from langchain.chains import RetrievalQA
answer_generation_chain = RetrievalQA.from_chain_type(llm = llm,chain_type="stuff",retriever = vector_store.as_retriever())

In [44]:
print(final_questions)

What is Flutter and what problem does it solve?
What are the key advantages of using Flutter for mobile app development, according to the tutorial?
What programming language is used in Flutter, and what prerequisites does the tutorial mention for someone learning Flutter?
Flutter is described as an open-source framework. What does open-source mean in this context, and why is it significant?
Explain the architecture of a Flutter application. What are the key layers involved?
What are widgets in Flutter? Give some examples of different types of widgets.
Explain the concept of "state" in Flutter. Why is state management important in Flutter applications?
What is the role of Gestures in Flutter and why is it important for application development?
Describe the difference between Single-child and multi-child layout widgets. Give examples of each.
Name some Single Child Widgets and explain what they do?
Name some Multiple Child Widgets and explain what they do?
Explain the difference between 

In [45]:
print(type(final_questions))

<class 'str'>


In [ ]:
for question in final_questions:
    print("QUESTION:",question)
    answer = answer_generation_chain.run(question)
    print("ANSWER:",answer)
    print("-------------------------------------------------------------------")
    with open("answers.txt","a") as file:
        file.write(question+"\n")
        file.write(answer+"\n")
        file.write("\n\n")
